In [44]:
from cytoolz     import assoc
from cytoolz     import assoc_in
from cytoolz     import first
from cytoolz     import get
from cytoolz     import get_in
from cytoolz     import merge
from cytoolz     import second
from cytoolz     import take
from cytoolz     import thread_first
from cytoolz     import thread_last
from firebird    import ARD
from firebird    import grid
from firebird    import pyccd
from firebird    import timeseries
from functools   import partial

from importlib   import reload
from merlin      import functions as f
from pyspark.sql import functions as ff
from pyspark.sql import SparkSession
from pyspark.sql import types

import firebird
import pyspark


In [ ]:
ss = SparkSession(pyspark.SparkContext.getOrCreate())

## Build timeseries & run PyCCD

In [ ]:
x = -2094585 
y = 1952805

tile = grid.tile(x=x, y=y, cfg=ARD)

chips = take(1, tile.get('chips'))

ids = timeseries.ids(ctx=ss.sparkContext, chips=chips)

ard = timeseries.rdd(ctx=ss.sparkContext, ids=ids, acquired='1980-01-01/2017-01-01', cfg=firebird.ARD, name='ard')

In [ ]:
rdd = pyccd.rdd(ctx=ss.sparkContext, timeseries=ard)
rdd.take(1)

## Retrieve matching aux data 

In [ ]:
aux = timeseries.aux(acquired='1980-01-01/2017-01-01', ctx=ss.sparkContext, ids=ids)

In [ ]:
aux.freqItems(cols=['trends']).collect()

In [ ]:
aux.first()

In [ ]:
aux.filter(aux.trends[0] != 1).first()

In [ ]:
ff.first(aux.trends)

In [ ]:
aux.trends[0]


In [ ]:
a1 = aux.filter('trends[0] NOT IN (1, 9)').select(aux.chipx, aux.chipy).distinct()
a2 = aux.filter('trends[0] NOT IN (2, 9)').select(aux.chipx, aux.chipy).distinct()
a3 = a1.join(a2, on=(['chipx', 'chipy']), how='inner')
a3.collect()

In [ ]:
g = get('snap_fn', firebird.AUX)
#firebird.AUX.get('snap_fn')(x=-2115585, y=1964805)
g(x=-2115585, y=1964805)

In [ ]:
tx, ty = get_in(['tile', 'proj-pt'], g(x=x, y=y))

## Get results off cluster onto local disk

In [ ]:
#it = aux.rdd.toLocalIterator()
#[print(i) for i in it]

## Write a User Defined Function

In [143]:
@ff.udf(returnType=types.StringType())
def myudf(trends, mpw, aspect, dem, posidex):
    return 'seperate'


@ff.udf(returnType=types.StringType())
def myudf2(*args, **kwargs):
    
    return str(args)

from pyspark.ml.linalg import Vector, VectorUDT

@ff.udf(returnType=VectorUDT())
def densify(data):
    #return Vectors.dense(data)
    return Vectors.dense(2)

@ff.udf(returnType=types.IntegerType())
def small(data):
    return data % 7

In [77]:
cols=['trends', 'mpw']
aux.withColumn('happiness', myudf2(*cols)).first()

Row(chipx=-2115585, chipy=1964805, x=-2114145, y=1964715, dates=[730332], dem=None, trends=[0], aspect=None, posidex=None, slope=None, mpw=None, happiness='([0], None)')

In [65]:
# find out how to select multiple columns programatically
ss.createDataFrame(aux.rdd.map(lambda x: x))

PythonRDD[80] at RDD at PythonRDD.scala:48

In [139]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StringIndexer


In [147]:
# vd = Vectors.dense(list(map(lambda x: fiyst(x) if type(x) in [tuple, list, set] else x, [[9, 9, 9], 1, 2, 3, 0, 1, 2])))

aa = aux.withColumn('features', densify(aux.trends))
bb = aa.withColumn('label', small(aux.chipx))

Help on method withColumnRenamed in module pyspark.sql.dataframe:

withColumnRenamed(existing, new) method of pyspark.sql.dataframe.DataFrame instance
    Returns a new :class:`DataFrame` by renaming an existing column.
    This is a no-op if schema doesn't contain the given column name.
    
    :param existing: string, name of the existing column to rename.
    :param col: string, new name of the column.
    
    >>> df.withColumnRenamed('age', 'age2').collect()
    [Row(age2=2, name='Alice'), Row(age2=5, name='Bob')]
    
    .. versionadded:: 1.3



In [145]:
from pyspark.ml.classification import RandomForestClassifier
r = RandomForestClassifier()


In [149]:
m = r.fit(bb)

In [153]:
m.write()

In [142]:
-100 % 2

0

In [155]:
from pyspark.ml import feature


In [157]:
bb.dtypes

[('chipx', 'int'),
 ('chipy', 'int'),
 ('x', 'int'),
 ('y', 'int'),
 ('dates', 'array<int>'),
 ('dem', 'array<float>'),
 ('trends', 'array<int>'),
 ('aspect', 'array<int>'),
 ('posidex', 'array<float>'),
 ('slope', 'array<float>'),
 ('mpw', 'array<int>'),
 ('features', 'vector'),
 ('label', 'int')]

In [158]:
types.

Help on function translate in module pyspark.sql.functions:

translate(srcCol, matching, replace)
    A function translate any character in the `srcCol` by a character in `matching`.
    The characters in `replace` is corresponding to the characters in `matching`.
    The translate will happen when any character in the string matching with the character
    in the `matching`.
    
    >>> spark.createDataFrame([('translate',)], ['a']).select(translate('a', "rnlt", "123") \
    ...     .alias('r')).collect()
    [Row(r='1a2s3ae')]
    
    .. versionadded:: 1.5

